# Import libraries

In [ ]:
!pip install transformers==4.32.0 accelerate tiktoken einops transformers_stream_generator==0.0.4 scipy torchvision pillow tensorboard matplotlib

In [ ]:
import os
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
torch.manual_seed(1)

# Instantiate model

In [ ]:
# Instantiate Qwen-VL-Chat model. via
# https://huggingface.co/Qwen/Qwen-VL-Chat
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map="cuda", trust_remote_code=True).eval()

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets/inferences', exist_ok=True)

# Import datasets

In [ ]:
constraint22_dataset_uspolitics_test = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/constraint22_dataset_uspolitics/constraint22_dataset_uspolitics_test.csv')
constrain22_dataset_covid19_test = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/constrain22_dataset_covid19/constrain22_dataset_covid19_test.csv')
DISARM_test_all = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/DISARM/DISARM_test_all.csv')
total_defense_memes = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/total_defense_memes/total_defense_memes.csv')

# Balanced sampling

In [ ]:
constraint22_dataset_uspolitics_test = constraint22_dataset_uspolitics_test.groupby('role').sample(n=250, random_state=1).reset_index(drop=True)
constrain22_dataset_covid19_test = constrain22_dataset_covid19_test.groupby('role').sample(n=190, random_state=1).reset_index(drop=True)

# Select unique images from **total_defense_memes**

In [ ]:
total_defense_memes = total_defense_memes[['image']].drop_duplicates().reset_index(drop=True)

# Set prompt and define a function to call the model

In [ ]:
# Use Qwen-VL-Chat for the inference. via
# https://huggingface.co/Qwen/Qwen-VL-Chat
def get_caption(image, entity):
    query = tokenizer.from_list_format([
        {'image': image},
        {'text': f'What does the meme show, in particular text and entities such as {entity}? Describe in english:'},
    ])
    response, history = model.chat(tokenizer, query=query, history=None)
    return response

In [ ]:
# Use Qwen-VL-Chat for the inference. via
# https://huggingface.co/Qwen/Qwen-VL-Chat
def get_caption_TDEF(image):
    query = tokenizer.from_list_format([
        {'image': image},
        {'text': 'What does the meme show, in particular text and entities? Describe in english:'},
    ])
    response, history = model.chat(tokenizer, query=query, history=None)
    return response

# Call the `get_caption` and `get_caption_TDEF` functions and save inferences

In [ ]:
uspolitics_test_images = constraint22_dataset_uspolitics_test['image'].values
uspolitics_test_entities = constraint22_dataset_uspolitics_test['entity'].values
constraint22_dataset_uspolitics_test['caption'] = [get_caption(image, entity) for image, entity in zip(uspolitics_test_images, uspolitics_test_entities)]
constraint22_dataset_uspolitics_test.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_Qwen-VL-Chat.csv', index=False)

In [ ]:
covid19_test_images = constrain22_dataset_covid19_test['image'].values
covid19_test_entities = constrain22_dataset_covid19_test['entity'].values
constrain22_dataset_covid19_test['caption'] = [get_caption(image, entity) for image, entity in zip(covid19_test_images, covid19_test_entities)]
constrain22_dataset_covid19_test.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constrain22_dataset_covid19_test_captioned_Qwen-VL-Chat.csv', index=False)

In [ ]:
DISARM_test_all_images = DISARM_test_all['image'].values
DISARM_test_all_entities = DISARM_test_all['target'].values
DISARM_test_all['caption'] = [get_caption(image, entity) for image, entity in zip(DISARM_test_all_images, DISARM_test_all_entities)]
DISARM_test_all.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_Qwen-VL-Chat.csv', index=False)

In [ ]:
total_defense_memes_images = total_defense_memes['image'].values
total_defense_memes['caption'] = [get_caption_TDEF(image) for image in total_defense_memes_images]
total_defense_memes.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/total_defense_memes_captioned_Qwen-VL-Chat.csv', index=False)